In [4]:
dataset = "DebtPenny_19930401_20221208.csv"

Install needed packages

In [3]:
import sys
!{sys.executable} -m pip install pandas_datareader

     |████████████████████████████████| 109 kB 4.3 MB/s            


In [4]:
!{sys.executable} -m pip install tensorflow

     |████████████████████████████████| 588.3 MB 20 kB/s              
     |████████████████████████████████| 2.4 MB 11.0 MB/s            
     |████████████████████████████████| 1.7 MB 15.2 MB/s            
     |████████████████████████████████| 77 kB 6.9 MB/s             
     |████████████████████████████████| 124 kB 12.1 MB/s            
     |████████████████████████████████| 14.1 MB 129 kB/s            
     |████████████████████████████████| 6.0 MB 2.5 MB/s             
     |████████████████████████████████| 65 kB 4.6 MB/s             
     |████████████████████████████████| 4.8 MB 10.5 MB/s            
     |████████████████████████████████| 439 kB 9.9 MB/s            
     |████████████████████████████████| 57 kB 4.9 MB/s             
     |████████████████████████████████| 4.9 MB 9.1 MB/s            
     |████████████████████████████████| 177 kB 14.5 MB/s            
     |████████████████████████████████| 93 kB 1.8 MB/s             
     |████████████████████████████████

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as reader
import datetime as datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.models import Sequential
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.utils.vis_utils import plot_model

In [ ]:
# qqq_data = pd.read_csv("QQQ.csv", index_col = "Date", parse_dates = True) #Read csv with pandas, parse dates and set date as index column
# qqq_data.head()

In [5]:
qqq_data = pd.read_csv(dataset) #Read csv with pandas, parse dates and set date as index column
qqq_data.head()

,Record Date,Debt Held by the Public,Intragovernmental Holdings,Total Public Debt Outstanding,Source Line Number,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
0,2022-12-08,2.448397e+13,6.826828e+12,3.131080e+13,1,2023,1,2022,4,12,8
1,2022-12-07,2.452278e+13,6.812309e+12,3.133509e+13,1,2023,1,2022,4,12,7
2,2022-12-06,2.452552e+13,6.813023e+12,3.133855e+13,1,2023,1,2022,4,12,6
3,2022-12-05,2.454506e+13,6.807741e+12,3.135280e+13,1,2023,1,2022,4,12,5
4,2022-12-02,2.454628e+13,6.803853e+12,3.135014e+13,1,2023,1,2022,4,12,2


In [11]:
qqq_data.dropna(how='all', inplace=True)

In [12]:
print(qqq_data.columns.values.tolist())

['Record Date', 'Debt Held by the Public', 'Intragovernmental Holdings', 'Total Public Debt Outstanding', 'Source Line Number', 'Fiscal Year', 'Fiscal Quarter Number', 'Calendar Year', 'Calendar Quarter Number', 'Calendar Month Number', 'Calendar Day Number']


In [13]:

x_train, x_test, y_train, y_test = train_test_split(qqq_data[['Debt Held by the Public', 'Intragovernmental Holdings', 'Source Line Number', 'Fiscal Year', 'Fiscal Quarter Number', 'Calendar Year', 'Calendar Quarter Number', 'Calendar Month Number', 'Calendar Day Number']], qqq_data['Total Public Debt Outstanding'], test_size=0.15, random_state=42)


In [14]:


print(x_train)
print(x_test)



      Debt Held by the Public  Intragovernmental Holdings  Source Line Number  \
6046                      NaN                         NaN                   1   
1782             1.330463e+13                5.227705e+12                   1   
2851             9.907753e+12                4.672952e+12                   1   
6956                      NaN                         NaN                   1   
1375             1.430530e+13                5.540655e+12                   1   
...                       ...                         ...                 ...   
5191                      NaN                         NaN                   1   
5226                      NaN                         NaN                   1   
5390                      NaN                         NaN                   1   
860              1.620245e+13                5.820374e+12                   1   
7270                      NaN                         NaN                   1   

      Fiscal Year  Fiscal Q

In [ ]:


#Model Architecture
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, x_train.shape[1]), activation="relu", return_sequences=False))
lstm.add(Dense(1))
#An optimizer is used to help improve upon the Loss function, while the Loss function gives us information on 
#how the model did on training.
lstm.compile(loss="mean_squared_error", optimizer="adam")



In [ ]:
plot_model(lstm, show_shapes=True, show_layer_names=True)

In [ ]:
xtrain = np.array(x_train)
xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1]) #Reshape data to align with parameter
#Here we start training the model by setting it to iterate over the entire dataset a 100 times, with a batch size of 8
history = lstm.fit(xtrain, y_train, epochs = 100, batch_size = 8, verbose = 1, shuffle = False)

In [ ]:
#Convert to a np array so that we can use it in our LSTM Model.
xtest = np.array(x_test)
#Now we need to reshape our data because our dataset is 2D but the LSTM Model expects a 3D shape
xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1]) #Reshape data to align with parameter
#Next we need to get the models predicted price values
model_pred = lstm.predict(xtest)
model_pred;

In [ ]:


#Actual Stock Prices
sns.lineplot(data=qqq_data, x="Date", y="Adj Close")



In [ ]:


#Predicted Stock Prices
x_test["Adj Close"] = model_pred
sns.lineplot(data=x_test, x="Date", y="Adj Close")



In [ ]:
#First we load in the data
riot_data = pd.read_csv("RIOT2016to2019.csv", index_col = "Date", parse_dates = True) #Read csv with pandas, parse dates and set date as index column
riot_data.head()

#Next we prepare the data
scale = MinMaxScaler(feature_range = (0, 1))
#Next we reshape the data to be able to fit the liner regression model
reshaped_data = riot_data['Close'].values.reshape(-1, 1)
scaling_data = scale.fit_transform(reshaped_data)
scaling_data;

In [ ]:
#Create empty arrays to store data to train
xtrain = []
ytrain = []
length_of_scale = len(scaling_data)
#We want to start prediction from 50 days to the end of the length of the scaled dataset
#We are going to fill the data by starting at the 50th index and iterate until the end of the scaled data
for values in range(50, length_of_scale):
    #Append values to the train dataset array, we are trying to set the model to train the first 50 values and append that to xtrain
    #Then from the 51st value that we already know we are going to set that to ytrain so that our model will be ble to predict the next values
    xtrain.append(scaling_data[values - 50 : values, 0])
    ytrain.append(scaling_data[values, 0])



In [ ]:


#Next we start convert the data to an array
xtrain = np.array(xtrain)
#Now we need to reshape our data because our dataset is 2D but the LSTM Model expects a 3D shape just like our previous model
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))
ytrain = np.array(ytrain) #Converting the data to an array
#To start building the Model, we will need to couple of things.
#We will need to build layers. We are going to create about 7 layers for our model
second_model = Sequential()
second_model.add(LSTM(40, input_shape = (xtrain.shape[1], 1), return_sequences = True))
#The Dropout layer is used to randomly sets input units to 0 with a frequency of rate
#at each step during training time. I am adding this so as to helps prevent overfitting.
second_model.add(Dropout(0.4))
#Adding another lader to the model to aid in prediction
second_model.add(LSTM(40, return_sequences = True))
#The Dropout layer is used to randomly sets input units to 0 with a frequency of rate
#at each step during training time. I am adding this so as to helps prevent overfitting.
second_model.add(Dropout(0.4))
second_model.add(LSTM(40))
second_model.add(Dropout(0.4))
#This will serve as the neuron that will predict the closing price value from our data after training
second_model.add(Dense(1))
#An optimizer is used to help imporove upon the Loss function, while the Loss function gives us information on 
#how the model did on training.
second_model.compile(loss="mean_squared_error", optimizer="adam")
second_model.fit(xtrain, ytrain, epochs = 30, batch_size = 25)



In [ ]:


#Get test data
test = pd.read_csv("RIOT2019to2021.csv", index_col = "Date", parse_dates = True) #Read csv with pandas, parse dates and set date as index column
test.head() #print out 
right_stock_price = test['Close'].values
#all_data = pd.read_csv("QQQ", index_col = "Date", parse_dates = True) #Read csv with pandas, parse dates and set date as index column
all_data = pd.concat((riot_data['Close'], test['Close']))
all_data.head() #print out 
all_data_length = len(all_data) #get the length so we know where to start
test_length = len(test) #get the length so we know where to start and end
data_sum = all_data_length - test_length #we know the exact length excess of both data
prediction_model = all_data[data_sum - 50 : ].values #we want to go from 1259 - 505 - the number of days to predict which is 50

#Next we need to scale it like before and all we are doing here is repeating the process above
prediction_model = prediction_model.reshape(-1, 1) #We need to reshape this model to column without changing its data

prediction_model_length = len(prediction_model)
prediction_model = scale.transform(prediction_model) #We need to transform because initially we scaled it up before. So now we have to turn it back to data we can use
#create new testing variable array
new_tester = []
#Now we can make predictions with all the data we have, which is what we did earlier, we are just repeating the step
for values in range(50, prediction_model_length):
  #Populate the array
  new_tester.append(prediction_model[values - 50 : values, 0])



In [ ]:
#After populating the array, next we 
new_tester = np.array(new_tester)
#Have to reshape new tester to fit the size of a 3D shape
new_tester = np.reshape(new_tester, (new_tester.shape[0], new_tester.shape[1], 1))

#After gathering all the info now we can predict the stock prices
potential_prices = second_model.predict(new_tester)
#potential_prices = scaler.inverse_transform(potential_prices)
revert_prices = scale.inverse_transform(potential_prices)
revert_prices;

In [ ]:
#Plot predictions
plt.plot(right_stock_price, color = 'green')
plt.plot(revert_prices, color = 'blue')
plt.xlabel("TIME SCALE", fontweight = "bold")
plt.ylabel("IPO PRINCE IN $", fontweight = "bold")



## LSTM COMPARISON

As you can see, this new LSTM model with more layers gave use a better price prediction since the Initial Public Offering 
of the QQQ Stock. Through Neural Networks, we set up predictions as to what the next price of the stock will be over 
time using historical data. On our second model, the Blue line serves as our predicted prices over the time scale. 
The green line on the plot serves as the actual price of the stock over the time scale. 
I believe it is safe to say our second approximation supersedes the first.

## THEOREM

We had a theory that the more layers and neurons that we used on our next model, the more successful our approximation 
would be. Through the stabilization, processing and analysis on our historic data, we created a collection of all the 
data by merging both year 2016 to 2019 and 2019 up until the present day. We had to create a set of data by gathering 
all the price fluctuations ever recorded since the stock IPO. That is exactly what we did by merging all the data. With that, 
we knew all the closing prices every recorded up until the present day. With all this data, we were ready to create the perfect neural network. We had all the data that we needed and we knew that there was a set optimal number of layers and neurons that our model needed to create the perfect configuration. As you can tell by our previous model, some of our predictions were overfitted. On this next model, we solved this problem by providing the model with the exact amount of neurons and layers that it needed to do the job. The result was the Neural Network predicting a better closing price estimation over the time scale.

## CONCLUSION

In conclusion, with the use of machine learning, we have successfuly analyzed and predicted the price of a stock across
specific times. This market prediction was derived with the analysis of Opening Value, Market High per day, Market Low per
day and Closing price per day. We also took into consideration the trading volume per day. With all these parameters factored 
in, we could get a pretty good esimation to what the actual stock prices were using all the above fearture parameters. 
Although this model yielded a pretty close estimation, the stock prices are steadily affected by the news about the company, 
activities going on in the country, the economy, news on mergers and collaborations, as well as news on pandemics. 
These factors can not be predicted ahead of time because of the nature of situations. Through Machine Learning algorithms, 
technology is able to saves us time and money in the sense that we can spend less time using procedures to compute and analyze 
market structures, as well as saving us money by creating models to predict the trends of markets which in turn will signify 
when it is a good time to enter a trade, or signify when to short a stock. Machine learning outperforms humans when it comes
to performance and speed. All this is made possible with the analysis of historical data.


In [ ]:


from google.colab import drive
drive.mount('/content/drive')

! pwd


%%shell
jupyter nbconvert --to html ///content/320Final.ipynb

